In [4]:
import pandas as pd
from IPython.display import display

# Load Datasets
movies = pd.read_csv('../data/cleaned_remapped_movies.csv')
ratings = pd.read_csv('../data/cleaned_remapped_ratings.csv')

display(movies.head(5))

,movieId,title,genres,year,genre_list,combined_features
0,415.0,toy story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,"['Adventure', 'Animation', 'Children', 'Comedy...",toy story (1995) Adventure|Animation|Children|...
1,191.0,jumanji (1995),Adventure|Children|Fantasy,1995,"['Adventure', 'Children', 'Fantasy']",jumanji (1995) Adventure|Children|Fantasy
2,941.0,grumpier old men (1995),Comedy|Romance,1995,"['Comedy', 'Romance']",grumpier old men (1995) Comedy|Romance
3,3313.0,waiting to exhale (1995),Comedy|Drama|Romance,1995,"['Comedy', 'Drama', 'Romance']",waiting to exhale (1995) Comedy|Drama|Romance
4,942.0,father of the bride part ii (1995),Comedy,1995,['Comedy'],father of the bride part ii (1995) Comedy


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english')

# Fit and transform the combined features
tfidf_matrix = tfidf.fit_transform(movies['combined_features'])

# Check the shape of the matrix
print("TF-IDF Matrix shape:", tfidf_matrix.shape)

TF-IDF Matrix shape: (87382, 42527)


In [8]:
from scipy.sparse import issparse

# Check if the matrix is sparse (Scipy format)
if issparse(tfidf_matrix):
    print("The matrix is sparse (Scipy format).")
else:
    # Calculate sparsity
    total_elements = matrix.size
    zero_elements = (matrix == 0).sum()
    sparsity = zero_elements / total_elements
    
    print(f"Sparsity: {sparsity:.2f}")
    if sparsity > 0.5:
        print("The matrix is sparse.")
    else:
        print("The matrix is dense.")

The matrix is sparse (Scipy format).


In [10]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Chunk size
chunk_size = 1000  # Adjust based on available memory
cosine_sim = np.zeros((tfidf_matrix.shape[0], tfidf_matrix.shape[0]))

# Compute in chunks
for start in range(0, tfidf_matrix.shape[0], chunk_size):
    # print(":::::::: => ", start)
    end = min(start + chunk_size, tfidf_matrix.shape[0])
    cosine_sim[start:end] = cosine_similarity(tfidf_matrix[start:end], tfidf_matrix)

print(" :::::: Cosine Similarity Shape => ", cosine_sim.shape)

:::::::: =>  0
:::::::: =>  1000
:::::::: =>  2000
:::::::: =>  3000
:::::::: =>  4000
:::::::: =>  5000
:::::::: =>  6000
:::::::: =>  7000
:::::::: =>  8000
:::::::: =>  9000
:::::::: =>  10000
:::::::: =>  11000
:::::::: =>  12000
:::::::: =>  13000
:::::::: =>  14000
:::::::: =>  15000
:::::::: =>  16000
:::::::: =>  17000
:::::::: =>  18000
:::::::: =>  19000
:::::::: =>  20000
:::::::: =>  21000
:::::::: =>  22000
:::::::: =>  23000
:::::::: =>  24000
:::::::: =>  25000
:::::::: =>  26000
:::::::: =>  27000
:::::::: =>  28000
:::::::: =>  29000
:::::::: =>  30000
:::::::: =>  31000
:::::::: =>  32000
:::::::: =>  33000
:::::::: =>  34000
:::::::: =>  35000
:::::::: =>  36000
:::::::: =>  37000
:::::::: =>  38000
:::::::: =>  39000
:::::::: =>  40000
:::::::: =>  41000
:::::::: =>  42000
:::::::: =>  43000
:::::::: =>  44000
:::::::: =>  45000
:::::::: =>  46000
:::::::: =>  47000
:::::::: =>  48000
:::::::: =>  49000
:::::::: =>  50000
:::::::: =>  51000
:::::::: =>  52000
::::::

In [12]:
# import pickle

# # Save cosine_sim to a .pkl file
# with open('../models/cosine_sim.pkl', 'wb') as f:
#     pickle.dump(cosine_sim, f)

# print(" :::::: Cosine similarity saved !!!!")
import h5py

# Save to HDF5 because this format supports batch processing for large datasets
with h5py.File('../models/cosine_sim.h5', 'w') as f:
    f.create_dataset('cosine_sim', data=cosine_sim, compression='gzip')

print(":::: HDF5 Saved !!!!")

:::: HDF5 Saved !!!!


In [82]:
import pickle

# # Save the cosine similarity matrix
# with open('data/recommendation/cosine_sim.pkl', 'wb') as f:
#     pickle.dump(cosine_sim, f)
# Save movie_indices after movieIds have been remapped later
# # Save the movie indices mapping
# with open('../models/movie_indices.pkl', 'wb') as f:
#     pickle.dump(movie_indices, f)